## Debug - Check User Collection Structure

# Intelligent Chatbot Routing System with LangGraph

This notebook implements an automated chatbot routing system that intelligently directs user queries to the appropriate agent without requiring manual selection.

## Architecture
- **Supervisor Agent**: Analyzes user queries and routes to appropriate sub-agent
- **Resume Agent**: Handles resume-related queries using FAISS vector DB
- **Reports Agent**: Handles interview reports and analytics queries using MongoDB
- **General Question Agent**: Handles general questions and conversation

## Database Integration
- **FAISS Vector DB**: For semantic search on user resumes
- **MongoDB**: For fetching interview reports and analytics

## ✅ Updates to Match Production Chatbot

This LangGraph implementation now matches the behavior of the deployed chatbot on the website:

### Reports Agent
- **Fetches detailed report types**: `verbal_reports`, `nonverbal_reports`, `overall_reports` collections
- **Passes FULL report data as JSON** to Gemini LLM
- **Extracts specific metrics**: WPM, pace, scores, feedback from structured report data
- **Example**: "What is my WPM?" → Returns exact value (81) from nonverbal report

### Resume Agent
- **Uses FAISS vector database** for semantic search on resume content
- **Returns specific information** from indexed resume chunks
- **Example**: "What programming languages do I know?" → Lists Python, Javascript, Java, Go

### General Agent
- **Uses SkillEdge-AI knowledge base** for platform information
- **Handles greetings and platform questions**
- **Example**: "What is SkillEdge-AI?" → Explains platform features

### Supervisor Agent
- **Intelligent routing** based on query intent
- **Routes WPM/performance metrics to reports agent** (not resume)
- **Recognizes patterns**: resume content, interview metrics, general questions

## ✅ Resume Query Fix - Matching Production Behavior

**Problem Solved**: The chatbot was incorrectly stating you know Java and Go.

### Root Cause
1. **Faulty skills extraction** in `resume_parser.py` used substring matching:
   - "**java**" matched inside "**Java**script"
   - "**go**" matched in "Lan**g**chain", "**go**al", etc.

2. **Using `top_k=5`** retrieved less relevant chunks including the faulty metadata chunk

### Solution Applied
1. **Updated `resume_parser.py`**: Now uses word boundary regex (`\b`) to match whole words only
2. **Changed to `top_k=3`**: Matches production setting, retrieves only most relevant chunks
3. **Improved prompt**: Simplified to match production's direct instruction style

### Test Results
✅ "which programming languages do i know?" → Returns "Python and Javascript"
✅ "do i know java?" → Returns "I couldn't find any mention of Java"  
✅ "do i know go?" → Returns "I couldn't find any mention of Go"

**Now matches production website behavior perfectly!**

## 🔄 Conversation Persistence

The chatbot now maintains **conversation history** across messages:

- **Stores message history** in `conversation_histories` dictionary
- **Provides context** to supervisor for better routing decisions
- **Maintains last 10 messages** for context window
- **Supports multiple conversations** via `conversation_id` parameter
- **Clear history** using `clear_conversation()` function

Example:
```python
# First message
await chat("What is my WPM?")  # Routes to reports agent → "Your WPM is 81"

# Follow-up message (maintains context)
await chat("How can I improve it?")  # Understands "it" = WPM → Routes to reports agent → Gives specific advice
```

## 1. Install Required Dependencies

## 2. Import Dependencies

In [ ]:
import os
import sys
from typing import TypedDict, Literal, Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
import operator
from dotenv import load_dotenv
import asyncio
from bson.objectid import ObjectId
import json

# Add parent directory to path to import app modules
sys.path.append(os.path.abspath("../.."))

# Load environment variables from .env file
load_dotenv(dotenv_path="../../.env")
print("✅ Environment variables loaded")

# Import resume RAG service and database functions
from app.chatbot.resume_rag import get_resume_rag_service
from app.database import (
    connect_to_mongo, 
    get_interview_reports_collection, 
    get_overall_reports_collection, 
    get_users_collection,
    get_verbal_reports_collection,
    get_nonverbal_reports_collection
)

print("✅ Imported resume RAG service and database modules")

## 3. Configuration

In [ ]:
# Load Gemini API key from .env file
GEMINI_API_KEY2 = os.getenv("GEMINI_API_KEY2")

if not GEMINI_API_KEY2:
    raise ValueError("GEMINI_API_KEY not found in .env file. Please add it to Backend/.env")

print(f"✅ API Key loaded: {GEMINI_API_KEY2[:10]}...")

# Set environment variable for Google
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY2

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,
    convert_system_message_to_human=True
)

print("✅ Gemini LLM initialized successfully!")

In [ ]:
# Initialize MongoDB connection
try:
    await connect_to_mongo()
    print("✅ MongoDB connected successfully!")
except Exception as e:
    print(f"❌ MongoDB connection failed: {e}")

# Initialize Resume RAG service
try:
    resume_rag = get_resume_rag_service()
    
    # Fix the index path to be relative to current working directory (notebook location)
    # Current: d:\Final Year Project\Backend\app\chatbot
    # Need: faiss_index/resumes (since we're already in app/chatbot)
    resume_rag.resume_index_path = "faiss_index/resumes"
    
    print("✅ Resume RAG service initialized!")
    print(f"📁 Resume index path: {os.path.abspath(resume_rag.resume_index_path)}")
except Exception as e:
    print(f"❌ Resume RAG initialization failed: {e}")

In [ ]:
# Check the structure of users in database
users_collection = get_users_collection()

# Search by _id (MongoDB ObjectId)
from bson import ObjectId
sample_user = await users_collection.find_one({"_id": ObjectId("69036f582c287be569c54623")})

if sample_user:
    print("✅ Found user in database!")
    print(f"\nUser document structure:")
    for key, value in sample_user.items():
        print(f"  {key}: {value}")
else:
    print("❌ User not found with _id: 69036f582c287be569c54623")
    print("\nLet's check what users exist:")
    all_users = await users_collection.find({}).limit(3).to_list(length=3)
    for i, user in enumerate(all_users, 1):
        print(f"\nUser #{i}:")
        print(f"  _id: {user.get('_id')}")
        print(f"  email: {user.get('email', 'N/A')}")

In [ ]:
# Debug: Check resume index paths and existence
user_id_to_check = "69036f582c287be569c54623"

print(f"Current working directory: {os.getcwd()}")
print(f"\nChecking resume index for user: {user_id_to_check}")
print(f"Resume RAG index base path: {resume_rag.resume_index_path}")

# Check if has_resume_index returns True
has_index = resume_rag.has_resume_index(user_id_to_check)
print(f"\nhas_resume_index() returned: {has_index}")

# Check actual file paths
index_path = resume_rag._get_user_index_path(user_id_to_check)
kb_path = resume_rag._get_user_kb_path(user_id_to_check)

print(f"\nExpected index file path: {index_path}")
print(f"Index file exists: {os.path.exists(index_path)}")

print(f"\nExpected KB file path: {kb_path}")
print(f"KB file exists: {os.path.exists(kb_path)}")

# Try with absolute path
abs_base = os.path.abspath("../../app/chatbot/faiss_index/resumes")
print(f"\nAbsolute base path: {abs_base}")
abs_index = os.path.join(abs_base, f"{user_id_to_check}_resume.index")
abs_kb = os.path.join(abs_base, f"{user_id_to_check}_resume_kb.pkl")
print(f"Absolute index path exists: {os.path.exists(abs_index)}")
print(f"Absolute KB path exists: {os.path.exists(abs_kb)}")

### Initialize Database Connection

## 4. Define State Schema

In [ ]:
class ChatbotState(TypedDict):
    """State schema for the chatbot graph"""
    messages: Annotated[list, add_messages]  # Chat history
    user_query: str  # Current user query
    route: str  # Which agent to route to
    response: str  # Final response to user
    user_id: str  # User identifier (for context)

## 5. Supervisor Agent - Query Analyzer & Router

In [ ]:
def supervisor_agent(state: ChatbotState) -> ChatbotState:
    """
    Supervisor agent that analyzes the user query and determines which sub-agent should handle it.
    
    Routes:
    - resume_agent: Questions about resume, skills, experience, education, projects
    - reports_agent: Questions about interview performance, analytics, scores, feedback, WPM, pace, etc.
    - general_agent: General conversation, greetings, other questions
    """
    
    user_query = state["user_query"]
    
    # System prompt for the supervisor
    supervisor_prompt = """You are an intelligent routing agent for a career interview platform chatbot.
Your job is to analyze the user's query and decide which specialized agent should handle it.

Available agents:
1. "resume" - Handles questions about:
   - User's resume content (skills, experience, education, projects)
   - Resume improvements and suggestions
   - Career profile information
   - Skills assessment
   - Professional background

2. "reports" - Handles questions about:
   - Interview performance and scores
   - Past interview reports and analytics
   - Interview feedback and recommendations
   - Performance trends and statistics
   - Speaking metrics (WPM, pace, speed, fluency, clarity)
   - Verbal analysis (answer quality, domain knowledge, correctness)
   - Non-verbal analysis (body language, confidence, voice)
   - Any specific performance data or metrics

3. "general" - Handles:
   - General conversation and greetings
   - Platform usage questions
   - Unrelated or unclear queries
   - Small talk

Important: Questions about performance metrics like WPM, pace, speed, scores, feedback should go to "reports" agent.

Analyze the query and respond with ONLY ONE WORD from: resume, reports, or general
Do not provide any explanation, just the routing decision."""
    
    # Get routing decision from LLM
    messages = [
        SystemMessage(content=supervisor_prompt),
        HumanMessage(content=f"User Query: {user_query}")
    ]
    response = llm.invoke(messages)
    
    # Extract and normalize the route
    route = response.content.strip().lower()
    
    # Validate route
    if route not in ["resume", "reports", "general"]:
        route = "general"  # Default fallback
    
    print(f"📍 Supervisor Decision: Routing to '{route}' agent")
    
    state["route"] = route
    state["messages"].append(AIMessage(content=f"[Routing to {route} agent]"))
    
    return state

## 6. Resume Agent

In [ ]:
async def resume_agent(state: ChatbotState) -> ChatbotState:
    """
    Resume agent handles queries about user's resume, skills, experience, and career profile.
    Uses FAISS vector DB for semantic search on actual resume data.
    """
    
    user_query = state["user_query"]
    user_id = state.get("user_id", "unknown")
    
    try:
        # Since there's no user_id field in users collection, use _id directly
        # The user_id passed in should be the MongoDB ObjectId string
        mongodb_id = user_id
        
        print(f"📌 Using MongoDB ID: {mongodb_id} for resume search")
        
        # Check if user has a resume indexed
        has_resume = resume_rag.has_resume_index(mongodb_id)
        
        if not has_resume:
            response_text = "I'd love to help you with your resume questions, but I don't see a resume indexed for your account yet. Please upload your resume first, and I'll be able to answer questions about your skills, experience, and projects!"
            print(f"\n📄 Resume Agent Response: {response_text}\n")
            state["response"] = response_text
            state["messages"].append(AIMessage(content=response_text))
            return state
        
        # Use top_k=3 to match production behavior
        # Gets the most relevant chunks without including less relevant metadata
        resume_results = await resume_rag.search_user_resume(mongodb_id, user_query, top_k=3)
        
        if not resume_results:
            response_text = "I couldn't find relevant information in your resume for that query. Could you rephrase your question?"
            print(f"\n📄 Resume Agent Response: {response_text}\n")
            state["response"] = response_text
            state["messages"].append(AIMessage(content=response_text))
            return state
        
        # Build context from search results
        context_parts = []
        for i, result in enumerate(resume_results, 1):
            context_parts.append(f"[Context {i}] {result['content']}")
        
        context = "\n\n".join(context_parts)
        
        print(f"✅ Found {len(resume_results)} relevant resume chunks")
        
        # System prompt matching production behavior
        resume_prompt = f"""You are SkillEdge-AI Assistant helping users with questions about their resume.

Information from the user's resume:
{context}

User Question: {user_query}

Guidelines:
- Answer based ONLY on the information provided above from their resume
- Be specific and reference actual content from the resume context
- Keep responses concise and to the point
- If the information isn't in the provided context, say you couldn't find it in their resume

Provide your answer:"""
        
        messages = [
            SystemMessage(content=resume_prompt),
            HumanMessage(content=user_query)
        ]
        response = llm.invoke(messages)
        
        answer = response.content
        print(f"\n📄 Resume Agent Response: {answer}\n")
        
        state["response"] = answer
        state["messages"].append(AIMessage(content=answer))
        
    except Exception as e:
        error_msg = f"I encountered an error while searching your resume: {str(e)}"
        print(f"\n❌ Resume Agent Error: {error_msg}\n")
        import traceback
        traceback.print_exc()
        state["response"] = error_msg
        state["messages"].append(AIMessage(content=error_msg))
    
    return state

## 7. Reports Agent

In [ ]:
async def reports_agent(state: ChatbotState) -> ChatbotState:
    """
    Reports agent handles queries about interview performance, analytics, and feedback.
    Uses MongoDB to fetch detailed verbal, nonverbal, and overall reports.
    Matches the behavior of the implemented chatbot in the website.
    """
    
    user_query = state["user_query"]
    user_id = state.get("user_id", "unknown")
    
    try:
        # Get all report collections (matching the implemented chatbot)
        verbal_collection = get_verbal_reports_collection()
        nonverbal_collection = get_nonverbal_reports_collection()
        overall_collection = get_overall_reports_collection()
        
        # Find latest verbal report
        verbal_report = await verbal_collection.find_one(
            {"user_id": user_id},
            sort=[("created_at", -1)]
        )
        
        # Find latest non-verbal report  
        nonverbal_report = await nonverbal_collection.find_one(
            {"user_id": user_id},
            sort=[("created_at", -1)]
        )
        
        # Find latest overall report
        overall_report = await overall_collection.find_one(
            {"user_id": user_id},
            sort=[("created_at", -1)]
        )
        
        # Check if user has any reports
        if not verbal_report and not nonverbal_report and not overall_report:
            response_text = "I don't see any interview reports for your account yet. Complete an interview to get personalized feedback and analytics!"
            print(f"\n📊 Reports Agent Response: {response_text}\n")
            state["response"] = response_text
            state["messages"].append(AIMessage(content=response_text))
            return state
        
        print(f"\n📊 Reports Agent - Found reports: Verbal={bool(verbal_report)}, NonVerbal={bool(nonverbal_report)}, Overall={bool(overall_report)}")
        
        # Helper function to serialize MongoDB documents
        def serialize_doc(doc):
            """Convert MongoDB document to JSON-serializable format"""
            if doc is None:
                return None
            serialized = {}
            for key, value in doc.items():
                if key == "_id":
                    continue  # Skip ObjectId
                elif hasattr(value, 'isoformat'):  # datetime
                    serialized[key] = value.isoformat()
                elif isinstance(value, dict):
                    serialized[key] = serialize_doc(value)
                elif isinstance(value, list):
                    serialized[key] = [serialize_doc(item) if isinstance(item, dict) else item for item in value]
                else:
                    serialized[key] = value
            return serialized
        
        # Build comprehensive context with FULL report data (like implemented chatbot)
        context_text = "User's Interview Reports:\n\n"
        
        if verbal_report:
            context_text += "=== VERBAL REPORT ===\n"
            context_text += json.dumps(serialize_doc(verbal_report), indent=2) + "\n\n"
        
        if nonverbal_report:
            context_text += "=== NON-VERBAL REPORT ===\n"
            context_text += json.dumps(serialize_doc(nonverbal_report), indent=2) + "\n\n"
        
        if overall_report:
            context_text += "=== OVERALL REPORT ===\n"
            context_text += json.dumps(serialize_doc(overall_report), indent=2) + "\n\n"
        
        # System prompt matching the implemented chatbot's behavior
        reports_prompt = f"""You are SkillEdge-AI Assistant, an intelligent chatbot for the SkillEdge-AI interview preparation platform.
You are analyzing user's interview performance reports.

Guidelines:
- Be helpful, professional, and encouraging
- Provide SPECIFIC answers using the EXACT data from the reports (scores, metrics, WPM, etc.)
- When user asks about specific metrics (like WPM, scores, pace), extract and report the EXACT values from the report data
- Be concise and to the point
- If the data doesn't contain what they're asking for, politely say so
- Focus on helping users improve their interview performance

User Query: {user_query}

{context_text}

Please provide a helpful and relevant response based on the report data above.

Important: If user asks about specific metrics (WPM, pace, scores, etc.), extract the EXACT values from the reports and provide them in your answer.
"""
        
        messages = [
            SystemMessage(content=reports_prompt),
            HumanMessage(content=user_query)
        ]
        response = llm.invoke(messages)
        
        answer = response.content
        print(f"\n📊 Reports Agent Response: {answer}\n")
        
        state["response"] = answer
        state["messages"].append(AIMessage(content=answer))
        
    except Exception as e:
        error_msg = f"I encountered an error while fetching your interview reports: {str(e)}"
        print(f"\n❌ Reports Agent Error: {error_msg}\n")
        import traceback
        traceback.print_exc()
        state["response"] = error_msg
        state["messages"].append(AIMessage(content=error_msg))
    
    return state

## 8. General Question Agent

In [ ]:
def general_agent(state: ChatbotState) -> ChatbotState:
    """
    General agent handles greetings, platform questions, and general conversation.
    Uses SkillEdge-AI knowledge base context.
    """
    
    user_query = state["user_query"]
    
    # SkillEdge-AI Knowledge Base (matching implemented chatbot)
    knowledge_base_context = """
SkillEdge-AI Platform Information:

- SkillEdge-AI is an advanced AI-powered interview simulation platform designed to help job seekers prepare for interviews.
- It provides comprehensive feedback on both verbal and non-verbal communication skills.
- Offers three main types of interviews: Technical (for programming/technical skills), Behavioral (for soft skills and personality fit), and Resume-based (focusing on professional background).
- Provides three detailed reports after each interview:
  * Verbal Report: Analyzes answer correctness, depth, domain knowledge
  * Non-Verbal Report: Examines speech patterns (WPM, pace, fluency), content quality, confidence level, communication clarity
  * Overall Report: Combines both analyses with actionable insights and improvement recommendations
- Uses advanced AI technologies including speech recognition, natural language processing, and machine learning.
- Helps improve interview performance by providing objective feedback on speaking pace, clarity, confidence, and content quality.
- Perfect for job seekers at any career level, students preparing for campus placements, professionals switching careers.
"""
    
    # System prompt for general agent
    general_prompt = f"""You are SkillEdge-AI Assistant, a friendly assistant for the SkillEdge-AI interview preparation platform.

Platform Information:
{knowledge_base_context}

Guidelines:
- Be helpful, professional, and encouraging
- Provide accurate information about SkillEdge-AI platform features and benefits
- If greeting, greet warmly
- If asking about platform, explain clearly using the information above
- Keep responses concise and conversational (2-3 sentences)
- If question is outside platform scope, politely acknowledge and redirect to platform features

User Question: {user_query}

Provide a helpful response:"""
    
    messages = [
        SystemMessage(content=general_prompt),
        HumanMessage(content=user_query)
    ]
    response = llm.invoke(messages)
    
    answer = response.content
    print(f"\n💬 General Agent Response: {answer}\n")
    
    state["response"] = answer
    state["messages"].append(AIMessage(content=answer))
    
    return state

## 9. Router Function - Conditional Edge

In [ ]:
def route_query(state: ChatbotState) -> Literal["resume_agent", "reports_agent", "general_agent"]:
    """
    Router function that determines which agent node to visit next based on supervisor's decision.
    """
    route = state["route"]
    
    if route == "resume":
        return "resume_agent"
    elif route == "reports":
        return "reports_agent"
    else:
        return "general_agent"

## 10. Build LangGraph Workflow

In [ ]:
# Create the graph with async support
workflow = StateGraph(ChatbotState)

# Add nodes (wrap async functions)
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("resume_agent", resume_agent)
workflow.add_node("reports_agent", reports_agent)
workflow.add_node("general_agent", general_agent)

# Set entry point
workflow.set_entry_point("supervisor")

# Add conditional edges from supervisor to sub-agents
workflow.add_conditional_edges(
    "supervisor",
    route_query,
    {
        "resume_agent": "resume_agent",
        "reports_agent": "reports_agent",
        "general_agent": "general_agent"
    }
)

# All sub-agents lead to END
workflow.add_edge("resume_agent", END)
workflow.add_edge("reports_agent", END)
workflow.add_edge("general_agent", END)

# Compile the graph
app = workflow.compile()

print("✅ LangGraph workflow compiled successfully with database integration!")

## 11. Visualize the Graph (Optional)

In [ ]:
# Try to visualize the graph
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Unable to visualize graph: {e}")
    print("Graph structure: supervisor -> [resume_agent | reports_agent | general_agent] -> END")

## 12. Test Function

In [ ]:
# Global conversation history storage
conversation_histories = {}

async def chat(user_query: str, user_id: str = "69036f582c287be569c54623", conversation_id: str = "default"):
    """
    Main chat function that processes user queries through the LangGraph workflow.
    Now with conversation history persistence!
    
    Args:
        user_query: The user's question or message
        user_id: MongoDB ObjectId (e.g., "69036f582c287be569c54623")
        conversation_id: Unique identifier for this conversation (maintains context)
    
    Returns:
        The chatbot's response
    """
    print(f"\n{'='*60}")
    print(f"🧑 User: {user_query}")
    print(f"{'='*60}")
    
    # Get or initialize conversation history for this conversation
    if conversation_id not in conversation_histories:
        conversation_histories[conversation_id] = []
    
    conversation_history = conversation_histories[conversation_id]
    
    # Add current user message to history
    conversation_history.append(HumanMessage(content=user_query))
    
    # Build context from last 10 messages for the supervisor
    context_messages = conversation_history[-10:]  # Last 10 messages
    
    # Build a context summary for the supervisor to understand previous conversation
    if len(conversation_history) > 1:
        recent_context = "\n".join([
            f"{'User' if isinstance(msg, HumanMessage) else 'Assistant'}: {msg.content[:100]}..."
            for msg in conversation_history[-4:-1]  # Last 3 exchanges (exclude current)
        ])
        context_aware_query = f"[Previous context: {recent_context}]\n\nCurrent query: {user_query}"
    else:
        context_aware_query = user_query
    
    # Initialize state with conversation history
    initial_state = {
        "messages": context_messages,
        "user_query": context_aware_query,  # Include context for better routing
        "route": "",
        "response": "",
        "user_id": user_id
    }
    
    # Run the workflow
    result = await app.ainvoke(initial_state)
    
    # Add assistant response to conversation history
    conversation_history.append(AIMessage(content=result['response']))
    
    print(f"{'='*60}")
    print(f"🤖 Final Response: {result['response']}")
    print(f"{'='*60}\n")
    
    return result["response"]

def clear_conversation(conversation_id: str = "default"):
    """Clear conversation history for a given conversation ID"""
    if conversation_id in conversation_histories:
        del conversation_histories[conversation_id]
        print(f"✅ Cleared conversation history for: {conversation_id}")

## 13. Test Cases - Resume Queries

In [ ]:
# # Test resume-related queries
# # Note: Replace "test_user_123" with an actual user ID that has a resume indexed
# await chat("What skills are mentioned in my resume?")

In [ ]:
# await chat("Can you tell me about my work experience?")

In [ ]:
# await chat("What projects have I worked on?")

## 14. Test Cases - Reports Queries

In [ ]:
# Test reports-related queries
# Note: Replace "test_user_123" with an actual user ID that has interview reports
# await chat("How did I perform in my last interview?")

In [ ]:
# Test WPM query (should now return specific data from nonverbal report)
# await chat("What is my WPM?")

In [ ]:
# Test general platform question
# await chat("What is SkillEdge-AI?")

In [ ]:
# Test resume query
# await chat("What programming languages do I know?")

In [ ]:
# # Test with top_k=3 (like production website)
# user_id = "69036f582c287be569c54623"
# query = "which programming languages do i know?"

# # Search with top_k=3 (production setting)
# results = await resume_rag.search_user_resume(user_id, query, top_k=3)

# print(f"Production setting (top_k=3) - Found {len(results)} chunks:\n")
# print("="*80)

# for i, result in enumerate(results, 1):
#     print(f"\n📄 Chunk {i} (Similarity: {result.get('similarity_score', 'N/A')}):")
#     print("-"*80)
#     print(result['content'])
#     print("-"*80)

In [ ]:
# Test with production query - "which programming languages do i know?"
# clear_conversation()  # Start fresh
# await chat("which programming languages do i know?")

In [ ]:
# Test negative case - verify it correctly says NO to Java and Go
# await chat("do i know java?")

In [ ]:
# Test Go as well
# await chat("do i know go?")

In [ ]:
# await chat("Show me my interview analytics")

In [ ]:
# await chat("What feedback did I receive on my interviews?")

## 15. Test Cases - General Queries

In [ ]:
# Test general queries
# await chat("Hello! How are you?")

In [ ]:
# await chat("What is this platform about?")

In [ ]:
# await chat("Thanks for your help!")

## 16. Interactive Testing

In [ ]:
# Interactive loop for testing
print("\n🎉 Interactive Chatbot with Database Integration (type 'quit' to exit)\n")
print("💡 Tip: Change user_id to test with different users\n")

current_user_id = "69036f582c287be569c54623"  # Change this to actual user ID

while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("👋 Goodbye!")
        break
    
    response = await chat(user_input, user_id=current_user_id)
    print()